In [86]:
from selenium.common.exceptions import NoSuchElementException    
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup
import datetime
import time
import os
import pandas as pd
import nltk

def returnChromeDriver(pathToChromeDriver):
    chrome_options = Options()
    chrome_options.add_experimental_option( "prefs",{'profile.managed_default_content_settings.javascript': 1, 'profile.managed_default_content_settings.images': 1, 'profile.managed_default_content_settings.stylesheet': 2} )
    chromedriver = pathToChromeDriver
    userAgent = "Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexus 5 Build/JOP40D) AppleWebKit/535.19 (KHTML, like Gecko) Chrome/18.0.1025.166 Mobile Safari/535.19"
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver, chrome_options = chrome_options)
    driver.maximize_window()
    return driver

def returnSearchUrl(question):
    baseGoogleQuery = "https://www.google.com/search?q="
    searchUrl = baseGoogleQuery + question.lower().replace(" ", "+").replace("?", "%3F").replace("'", "%27")
    return searchUrl

def clickQuestions(driver, question, totalClicks):
    searchUrl = returnSearchUrl(question)
    driver.get(searchUrl)
    time.sleep(1)

    if driver.find_elements_by_css_selector('div.related-question-pair'):
        questionIndex = 0
        questions = driver.find_elements_by_css_selector('div.related-question-pair')
        
        itr = 0
        
        while itr < totalClicks:
            
            questions[itr].click()
            time.sleep(1)
            questions = driver.find_elements_by_css_selector('div.related-question-pair')
            itr += 1

    return driver

def extractQuestionData(soup):
    question_data = []
    
    for question in soup.findAll("div", class_="related-question-pair"):
        
        answers = question.find_all("div", id = lambda value: value and value.startswith("WEB_ANSWERS_RESULT"))
        for answer in answers:
            if answer.find("span") and answer.find("span").find("span"):
                a = answer.find("span").find("span")
                question_data.append((question.find("div")["data-q"], str(a)[str(a).find(">") + 1:str(a).rfind("<")], a.text))
                
    return question_data


def get_answers(questions, num_questions = 10, pathToChromeDriver = "C:/Users/Nishant/chromedriver/chromedriver"):
    
    driver = returnChromeDriver(pathToChromeDriver)
    
    all_data = []
    
    for question in questions:
        
        try:
        
            driver = clickQuestions(driver, question, num_questions)
            if driver.find_elements_by_css_selector('div.related-question-pair'):
                soup = BeautifulSoup(driver.page_source, "lxml")
                extractedData = extractQuestionData(soup)

                all_data.extend(extractedData)

            else:
                print("No Questions Found For: " + question)
                
        except:
            asdf = 1
            
    return all_data

    driver.quit()

In [76]:
df = pd.read_csv("Keywords.csv")
keywords = df["machine learning"]
pos_questions = [f"What is the history of {kw}?" for kw in keywords[:50]]

In [77]:
res = get_answers(question_inputs, 5)

C:\Users\Nishant\AppData\Local\Temp\ipykernel_3148\2396248721.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver, chrome_options = chrome_options)
C:\Users\Nishant\AppData\Local\Temp\ipykernel_3148\2396248721.py:16: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chromedriver, chrome_options = chrome_options)
C:\Users\Nishant\AppData\Local\Temp\ipykernel_3148\2396248721.py:30: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  if driver.find_elements_by_css_selector('div.related-question-pair'):
C:\Users\Nishant\AppData\Local\Temp\ipykernel_3148\2396248721.py:32: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  questions = driver.find_elements_by_css_selector('div.related-question-pair')

In [80]:
pos_data = res

In [82]:
neg_questions = [f"What is {kw}?" for kw in keywords[:50]]

In [83]:
neg_data = get_answers(neg_questions, 5)

C:\Users\Nishant\AppData\Local\Temp\ipykernel_3148\2396248721.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver, chrome_options = chrome_options)
C:\Users\Nishant\AppData\Local\Temp\ipykernel_3148\2396248721.py:16: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chromedriver, chrome_options = chrome_options)
C:\Users\Nishant\AppData\Local\Temp\ipykernel_3148\2396248721.py:30: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  if driver.find_elements_by_css_selector('div.related-question-pair'):
C:\Users\Nishant\AppData\Local\Temp\ipykernel_3148\2396248721.py:32: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  questions = driver.find_elements_by_css_selector('div.related-question-pair')

In [109]:
data = []

for q, answer_html, answer_text in pos_data:
    if "machine learning" in q.lower():
        continue
    
    sentences_html = nltk.sent_tokenize(answer_html)
    sentences_text = nltk.sent_tokenize(answer_text)
    
    q_lower = q.lower()
    
    tags = ["history", "invent", "who", "where", "first", "past", "future", "father", "when", "origin", "found", "oldest", "newest", "start"]
    is_instance = False
    for tag in tags:
        if tag in q_lower:
            is_instance = True
            break
    
    for i in range(min(len(sentences_html), len(sentences_text))):
        if "<b>" in sentences_html[i]:
            data.append((sentences_text[i], is_instance))

In [115]:
inst_df = pd.DataFrame(data, columns = ["sentence", "is_instance"])
inst_df.to_csv("people_ask.csv")

In [117]:
inst_df.sample(n = 5)

,sentence,is_instance
476,When Business Process Management (BPM) emerged...,True
159,Univariate feature selection works by selectin...,False
354,Augmented reality (AR) is an experience where ...,False
636,"ohn McCarthy, father of artificial intelligenc...",True
44,Data mining began in the 1990s and is the proc...,True
